# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233140230195                   -0.50    7.0
  2   -7.249590389826       -1.78       -1.41    1.0
  3   -7.251259213572       -2.78       -1.89    4.0
  4   -7.251286368126       -4.57       -2.16    3.0
  5   -7.251333613419       -4.33       -2.68    3.0
  6   -7.251337683289       -5.39       -3.13    1.0
  7   -7.251338698557       -5.99       -3.64    1.0
  8   -7.251338789803       -7.04       -4.12    2.0
  9   -7.251338798322       -8.07       -4.82    2.0
 10   -7.251338798677       -9.45       -5.35    2.0
 11   -7.251338798699      -10.66       -5.67    3.0
 12   -7.251338798704      -11.29       -6.36    2.0
 13   -7.251338798704      -12.61       -6.62    3.0
 14   -7.251338798705      -13.30       -6.98    1.0
 15   -7.251338798705      -14.01       -7.71    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.09247704319739801
[ Info: Arnoldi iteration step 2: normres = 0.46556568479834864
[ Info: Arnoldi iteration step 3: normres = 0.4984631665119583
[ Info: Arnoldi iteration step 4: normres = 0.39440252488897715
[ Info: Arnoldi iteration step 5: normres = 0.2642604637780276
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.34e-03, 7.54e-02, 1.67e-01, 1.84e-01, 4.87e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5031608728216457
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.21e-03, 1.58e-01, 4.61e-01, 9.34e-02, 3.53e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08774758737339433
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.08e-04, 2.89e-02, 2.37e-02, 2.68e-02, 6.88e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11632959240708601
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.47e